# Correction on PCA

In [4]:
%loadlibs
from utils import load_datasets, rnmse
import sklearn
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, TensorDataset

Loaded libraries:
	- numpy (np)
	- matplotlib.pyplot (plt)
	- torch
	- torch.nn (nn)
	- torch.optim (optim)
	- tqdm


In [1]:
device = 'mps'
x = torch.cat([torch.load("dataset/x_train.pt"), torch.load("dataset/x_train_2.pt")])
y = torch.cat([torch.load("dataset/y_train.pt"), torch.load("dataset/y_train_2.pt")])
c = torch.cat([torch.load("dataset/c_train.pt"), torch.load("dataset/c_train_2.pt")])

x_test = torch.load("dataset/x_test.pt")
y_test = torch.load("dataset/y_test.pt")
c_test = torch.load("dataset/c_test.pt")
x, y, c, x_test, y_test, c_test = x.numpy(), y.numpy(), c.numpy(), x_test.numpy(), y_test.numpy(), c_test.numpy()

seismogram_shape = x.shape[1:]

x = x.reshape((x.shape[0], -1))
y = y.reshape((y.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))
y_test = y_test.reshape((y_test.shape[0], -1))

pca1 = PCA(n_components=1024)
pca2 = PCA(n_components=1024)

pca1 = pca1.fit(x)
pca2 = pca2.fit(y)

Loaded libraries:
	- numpy (np)
	- matplotlib.pyplot (plt)
	- torch
	- torch.nn (nn)
	- torch.optim (optim)
	- tqdm


In [13]:
nx, nt = 128, 256
n_epochs = 500
l_r = 5e-5

In [14]:
x_pca, y_pca = pca1.transform(x), pca2.transform(y)

In [15]:
x_pca_max, x_pca_min, y_pca_max, y_pca_min = x_pca.max(), x_pca.min(), y_pca.max(), y_pca.min()
x_pca = (x_pca-x_pca_min)/(x_pca_max-x_pca_min)
y_pca = (y_pca-y_pca_min)/(y_pca_max-y_pca_min)
x_pca, y_pca = torch.tensor(x_pca), torch.tensor(y_pca)
train = DataLoader(TensorDataset(x_pca, y_pca), batch_size=32, shuffle=True)

In [ ]:
class CorrectionMLPPCA(nn.Module):
    def __init__(self, nx, hidden=512):
        super().__init__()
        self.xt_block = nn.Sequential(
            nn.Linear(nx, hidden*2),
            nn.Tanh(),
            nn.Linear(hidden*2, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden*2),
            nn.Tanh(),
            nn.Linear(hidden*2, nx),
        )

    def forward(self, x):
        y = self.xt_block(x)
        return y + x

model = CorrectionMLPPCA(1024, hidden=256).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=l_r)
best_val_loss = 10
for epoch in (pbar:=tqdm(range(n_epochs))):
    model.train()
    losses = []
    for xb, yb in train:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = rnmse(pred, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
        pbar.set_postfix(loss=np.mean(losses[-20:]))

  0%|          | 0/500 [00:00<?, ?it/s]